In [71]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


In [72]:
data_raw = pd.read_csv('../../DATA/FINAL/transformed_set_enriched.csv')

In [73]:
KEYS = [
    'entity_id',
    'financial_report_id',
    'financial_statement_id',
    'entity_ico',
    'entity_name']

DIMENSIONS = [
    'year',
    # 'month',
    # 'financial_report_order_number',
    'period_length',
    # 'consolidated',
    # 'approved_date',
    'type',
    'financial_report_template_id',
    # 'establishment_date',
    # 'cancellation_date',
    # 'sknace_code',
    # 'entity_state',
    'legal_form_code',
    'org_size_code',
    'ownership_category_code',
    'entity_consolidated',
    # 'sknace_division_name',
    'sknace_division',
    # 'sknace_subcategory',
    'sal_class',
    'is_outlier']

RAW_VALUES = [ 
    'total_assets',
    'non_current_assets',
    'non_current_intangible_assets',
    'non_current_tangible_assets',
    'non_current_financial_assets',
    'current_assets',
    'inventories',
    'non_current_receivables',
    'current_receivables',
    'financial_assets',
    'current_financial_assets',
    'cash_and_bank_balances',
    'total_equity_and_liabilities',
    'equity',
    'share_capital',
    'share_premium',
    'net_profit_of_previous_years',
    'net_profit_after_tax',
    'liabilities',
    'non_current_liabilities',
    'long_term_provisions',
    'long_term_bank_loans',
    'current_liabilities',
    'short_term_provisions',
    'current_bank_loans',
    'short_term_financial_assistance',
    'operating_revenues',
    'sales_from_the_merchandise',
    'sales_from_the_own_products_and_services',
    'sales_from_the_other',
    'other_revenues_from_operating_activities',
    'operating_costs',
    'costs_of_merchandise_sold',
    'consumed_materials_energy_non_inventory_supplies',
    'services',
    'personnel_costs',
    'taxes_and_fees',
    'depreciation',
    'remaining_cost_of_sold_long_term_assets_and_materials',
    'adjustments_to_receivables',
    'other_costs_of_operating_activities',
    'operating_result',
    'value_added',
    'revenues_from_financial_activities',
    'cost_of_financial_activities',
    'result_from_fincancial_activities',
    'result_before_tax',
    'income_tax',
    'result_after_tax']

PRIMARY_VALUES = [
    'CA',
    'CASH',
    'CL',
    'EQ',
    'NCL',
    'TA',
    'TL',
    'CC',
    'INV',
    'CF_NETTO',
    'CF_SELFFIN',
    'EAT',
    'EBIT',
    'SAL',
    'COST',
    'INT',
    'YIE',
    'LABOR']

RATIOS = [
    'L3',
    'L2',
    'L1',
    'CF_CL',
    'CASH_TA',
    'SAL_TA',
    'TL_SAL',
    'INV_COST',
    'INV_SAL',
    'CC_SAL',
    'TA_SAL',
    'TL_TA',
    'CF_TL',
    'CL_TA',
    'NCL_TA',
    'EQ_TL',
    'EQ_TA',
    'EBIT_INT',
    'CL_CC',
    'ROE',
    'EAT_TA',
    'ROA_BRUTTO',
    'CF_TA',
    'CF_SAL',
    'ROS',
    'EAT_YIE',
    'ROI',
    'ROA_NETTO',
    'LABOR_PRODUCTIVITY']

DISTRESS_SIGNALS = [
    'DPHZ_vat_registration_cancelled',
    'DPHZ_vat_registration_cancelled_1y_off',
    'DPHZ_vat_registration_cancelled_2y_off',
    'DPHZ_vat_registration_cancelled_3y_off',
    'RU_event',
    'RU_event_1y_off',
    'RU_event_2y_off',
    'RU_event_3y_off',
    'cancellation_year',
    'RUZ_cancelled',
    'RUZ_cancelled_1y_off',
    'RUZ_cancelled_2y_off',
    'RUZ_cancelled_3y_off']

In [74]:
data = data_raw.drop(columns=RAW_VALUES)
data_raw = None
display(data.shape)

(2114837, 86)

In [75]:
data = data.query('ownership_category_code in [2, 7, 8] and sknace_division in ["G","M","C","F","N","L","J","H","Q","I","A"] and establishment_date <= "2013-01-01"')

In [76]:
# pomer VI a zavazkov je mensi ako 8, t.j EQ_TL < 0.08
# 2016 : 0.04, 2017 : 0.06, 2018 : 0.08, 
# EQ < 0
# L3 < 1


def process_issues(row):
    EQ_TL = row.EQ_TL
    EQ = row.EQ
    L3 = row.L3
    EAT = row.EAT
    y = row.year

    if y <= 2016: threat_criterion = 0.04
    elif y == 2017: threat_criterion = 0.06
    else: threat_criterion = 0.08

    if EQ_TL < threat_criterion and EQ < 0 and L3 < 1 and EAT < 0:
        return 1
    return 0
    

In [77]:
ruz_indirect_list = []

for _, row in data.iterrows():
    criteria_value = process_issues(row)
    ruz_indirect_list.append([row['entity_id'], row['year'], row['period_length'], criteria_value])

ruz_indirect_df = pd.DataFrame(ruz_indirect_list, columns=['entity_id', 'year', 'period_length', 'RUZ_indirect_criteria'])


In [78]:
ruz_indirect_df['period_length_check'] = ruz_indirect_df['period_length'].apply(lambda x: 1 if x == 12 else 0)
ruz_indirect_df

ruz_indirect_df = (ruz_indirect_df
                   .sort_values(['period_length_check', 'RUZ_indirect_criteria'], ascending=[False, False])
                   .groupby(['entity_id', 'year'])
                   .first()
                   .reset_index())[['entity_id', 'year', 'RUZ_indirect_criteria']]

In [79]:
def process_entity_issues_2y_off(row, issue_type):
        return max(row[issue_type+'_1y_off'], row[issue_type+'_2y_off'])

def process_entity_issues_3y_off(row, issue_type):
        return max(row[issue_type+'_1y_off'], row[issue_type+'_2y_off'], row[issue_type+'_3y_off'])


entity_issues_indicators_1y_off = ruz_indirect_df.copy()
entity_issues_indicators_2y_off = ruz_indirect_df.copy()
entity_issues_indicators_3y_off = ruz_indirect_df.copy()


entity_issues_indicators_1y_off['year'] = entity_issues_indicators_1y_off['year'] - 1
entity_issues_indicators_1y_off.rename(columns={'RUZ_indirect_criteria':'RUZ_indirect_criteria_1y_off'}, inplace=True)

entity_issues_indicators_2y_off['year'] = entity_issues_indicators_2y_off['year'] - 2
entity_issues_indicators_2y_off.rename(columns={'RUZ_indirect_criteria':'RUZ_indirect_criteria_2y_off'}, inplace=True)

entity_issues_indicators_3y_off['year'] = entity_issues_indicators_3y_off['year'] - 3
entity_issues_indicators_3y_off.rename(columns={'RUZ_indirect_criteria':'RUZ_indirect_criteria_3y_off'}, inplace=True)


entity_issues_indicators = (ruz_indirect_df
                            .merge(
                                entity_issues_indicators_1y_off,
                                on = ['entity_id', 'year'],
                                how = 'outer'
                            )                           
                            .merge(
                                entity_issues_indicators_2y_off,
                                on = ['entity_id', 'year'],
                                how = 'outer'
                            )                                                    
                            .merge(
                                entity_issues_indicators_3y_off,
                                on = ['entity_id', 'year'],
                                how = 'outer'
                            ))

for col in ['RUZ_indirect_criteria',
       'RUZ_indirect_criteria_1y_off', 'RUZ_indirect_criteria_2y_off',
       'RUZ_indirect_criteria_3y_off']:
       entity_issues_indicators[col] = entity_issues_indicators[col].fillna(0)

entity_issues_indicators['RUZ_indirect_criteria_2y_off'] = entity_issues_indicators.apply(lambda x: process_entity_issues_2y_off(x, 'RUZ_indirect_criteria'), axis = 1)
entity_issues_indicators['RUZ_indirect_criteria_3y_off'] = entity_issues_indicators.apply(lambda x: process_entity_issues_3y_off(x, 'RUZ_indirect_criteria'), axis = 1)

entity_issues_indicators


,entity_id,year,RUZ_indirect_criteria,RUZ_indirect_criteria_1y_off,RUZ_indirect_criteria_2y_off,RUZ_indirect_criteria_3y_off
0,4.0,2014.0,0.0,0.0,0.0,0.0
1,4.0,2015.0,0.0,0.0,0.0,0.0
2,4.0,2016.0,0.0,0.0,0.0,0.0
3,4.0,2017.0,0.0,0.0,0.0,0.0
4,4.0,2018.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1380526,1598000.0,2012.0,0.0,0.0,0.0,0.0
1380527,1611042.0,2011.0,0.0,0.0,0.0,0.0
1380528,1611565.0,2011.0,0.0,0.0,0.0,0.0
1380529,1635085.0,2012.0,0.0,0.0,0.0,1.0


In [80]:
data = data.merge(entity_issues_indicators, on = ['entity_id', 'year'], how = 'left')

In [81]:
DISTRESS_SIGNALS = [
    'DPHZ_vat_registration_cancelled',
    'DPHZ_vat_registration_cancelled_1y_off',
    'DPHZ_vat_registration_cancelled_2y_off',
    'DPHZ_vat_registration_cancelled_3y_off',
    'RU_event',
    'RU_event_1y_off',
    'RU_event_2y_off',
    'RU_event_3y_off',
    'cancellation_year',
    'RUZ_cancelled',
    'RUZ_cancelled_1y_off',
    'RUZ_cancelled_2y_off',
    'RUZ_cancelled_3y_off',
    'RUZ_indirect_criteria',
    'RUZ_indirect_criteria_1y_off',
    'RUZ_indirect_criteria_2y_off',
    'RUZ_indirect_criteria_3y_off']


In [82]:
def get_sample_sizes(data, row_label, column_label):
    samples_df = data.groupby([row_label, column_label])['financial_statement_id'].nunique().to_frame('count').reset_index()
    wide_df = pd.pivot_table(samples_df, index = row_label, columns=column_label, fill_value=0)
    return wide_df

In [83]:
def combined_criteria(row, years_offset):
    return max(row[f'RU_event_{int(years_offset)}y_off'], row[f'RUZ_cancelled_{int(years_offset)}y_off'], row[f'RUZ_indirect_criteria_{int(years_offset)}y_off'])

data['combined_issue_criteria_1y_off'] = data.apply(lambda x: combined_criteria(x, 1), axis = 1)
data['combined_issue_criteria_2y_off'] = data.apply(lambda x: combined_criteria(x, 2), axis = 1)
data['combined_issue_criteria_3y_off'] = data.apply(lambda x: combined_criteria(x, 3), axis = 1)

data

,entity_id,year,month,financial_report_id,financial_report_template_id,CA,CASH,CL,EQ,NCL,TA,TL,CC,INV,CF_NETTO,CF_SELFFIN,EAT,EBIT,SAL,COST,INT,YIE,LABOR,L3,L2,L1,CF_CL,CASH_TA,SAL_TA,TL_SAL,INV_COST,INV_SAL,CC_SAL,TA_SAL,TL_TA,CF_TL,CL_TA,NCL_TA,EQ_TL,EQ_TA,EBIT_INT,CL_CC,ROE,EAT_TA,ROA_BRUTTO,CF_TA,CF_SAL,ROS,EAT_YIE,ROI,ROA_NETTO,LABOR_PRODUCTIVITY,financial_statement_id,financial_report_order_number,entity_ico,period_length,consolidated,approved_date,type,entity_name,establishment_date,cancellation_date,sknace_code,entity_state,legal_form_code,org_size_code,ownership_category_code,entity_consolidated,sknace_division_name,sknace_division,sknace_subcategory,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,DPHZ_vat_registration_cancelled_3y_off,RU_event,RU_event_1y_off,RU_event_2y_off,RU_event_3y_off,cancellation_year,RUZ_cancelled,RUZ_cancelled_1y_off,RUZ_cancelled_2y_off,RUZ_cancelled_3y_off,sal_class,is_outlier,RUZ_indirect_criteria,RUZ_indirect_criteria_1y_off,RUZ_indirect_criteria_2y_off,RUZ_indirect_criteria_3y_off,combined_issue_criteria_1y_off,combined_issue_criteria_2y_off,combined_issue_criteria_3y_off
0,4.0,2014.0,1.0,4644773,699.0,8787618.0,5238983.0,4658457.0,2112122.0,3232.0,11200988.0,8657266.0,2578606.0,16429.0,1171078.0,1063283.0,1007897.0,1115692.0,18580340.0,18172171.0,4446.0,19361305.0,3384185.0,1.886380,1.678150,1.124618,0.251388,0.467725,1.658813,0.465937,0.325467,0.318317,0.138781,217.022707,0.772902,0.135271,0.415897,0.000289,0.243971,0.188566,251.942870,1.806580,0.477196,0.089983,0.099607,0.104551,0.063028,0.054245,0.052057,0.090380,0.089983,0.182138,2382229.0,1.0,603783.0,12.0,NaN,2015-07-07,Riadna,"NESS Slovensko, a.s.",1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,2015.0,1.0,5382540,699.0,20276962.0,5687157.0,6153737.0,13229589.0,2890.0,20862929.0,7184264.0,14037315.0,6792.0,3940396.0,3245262.0,6536244.0,7231378.0,17608945.0,11551924.0,2910.0,29573853.0,3581817.0,3.295065,3.205284,0.924179,0.640326,0.272596,0.844030,0.407989,0.211663,0.138857,0.797170,426.524953,0.344355,0.548476,0.294960,0.000139,1.841468,0.634119,2486.009622,0.438384,0.494063,0.313295,0.346614,0.188871,0.223772,0.371189,0.221014,0.313434,0.313295,0.203409,2968194.0,1.0,603783.0,12.0,NaN,2016-09-28,Riadna,"NESS Slovensko, a.s.",1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,2016.0,1.0,5665728,699.0,5991927.0,1828391.0,4859422.0,1178778.0,3568.0,6518310.0,5120402.0,3840942.0,5894.0,247140.0,11702.0,-44282.0,191156.0,12878215.0,12564620.0,38229.0,12914606.0,3476051.0,1.233053,1.166668,0.376257,0.050858,0.280501,1.975698,0.397602,0.168874,0.164762,0.298251,182.214041,0.785541,0.048266,0.745503,0.000547,0.230212,0.180841,6.000288,1.265164,-0.037566,-0.006793,0.029326,0.037915,0.019191,-0.003439,-0.003429,-0.000929,-0.006793,0.269917,3194585.0,3.0,603783.0,12.0,NaN,2017-06-12,Riadna,"NESS Slovensko, a.s.",1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,2017.0,1.0,6245237,699.0,3016170.0,1224056.0,2649399.0,268147.0,3224.0,3442598.0,2902134.0,1355249.0,1954.0,-1327325.0,-1211994.0,-1270731.0,-1386062.0,7651564.0,9093625.0,9037.0,7725983.0,3304021.0,1.138436,0.973543,0.462013,-0.500991,0.355562,2.222613,0.379286,0.077355,0.091934,0.177121,161.971498,0.843007,-0.457362,0.769593,0.000937,0.092396,0.077891,-152.376342,1.954917,-4.738934,-0.369120,-0.402621,-0.385559,-0.173471,-0.166075,-0.164475,-0.366495,-0.369120,0.431810,3662172.0,2.0,603783.0,12.0,NaN,2018-06-26,Riadna,"NESS Slovensko, a.s.",1991-01-17,NaN,62090.0,NaN,121.0

In [86]:
temp = data.query('period_length == 12')[['entity_id', 'year', 'cancellation_year', 'RU_event_3y_off', 'RUZ_cancelled_3y_off']]

In [93]:
plausible_entity_ids = pd.DataFrame()

for current_year in 2015, 2016, 2017, 2018:
    previous_year = current_year - 1
    two_consequent_years = set(temp.query('year == @current_year')['entity_id']).intersection(temp.query('year == @previous_year')['entity_id'])

    one_off = current_year + 1
    two_off = current_year + 2
    three_off = current_year + 3

    all_three = (set(temp.query('year == @one_off')['entity_id'])
                 .intersection(temp.query('year == @two_off')['entity_id'])
                 .intersection(temp.query('year == @three_off')['entity_id']))
    
    two_and_cancelled = (set(temp.query('year == @one_off')['entity_id'])
                         .intersection(temp.query('year == @two_off')['entity_id'])
                         .intersection(temp.query('cancellation_year == @three_off')['entity_id']))
    
    one_and_cancelled = (set(temp.query('year == @one_off')['entity_id'])
                         .intersection(temp.query('cancellation_year == @two_off')['entity_id']))
    
    next_cancelled = set(temp.query('cancellation_year == @one_off')['entity_id'])

    ru_event_pass = set(temp.query('year == @current_year and RU_event_3y_off == 1 or RUZ_cancelled_3y_off == 1')['entity_id'])

    future_set = (all_three
                  .union(two_and_cancelled)
                  .union(one_and_cancelled)
                  .union(next_cancelled)
                  .union(ru_event_pass))
    
    total = two_consequent_years.intersection(future_set)

    yearly_plausible_entity_ids = pd.DataFrame(total, columns = ['entity_id'])
    yearly_plausible_entity_ids['year'] = current_year
    
    plausible_entity_ids = pd.concat([plausible_entity_ids, yearly_plausible_entity_ids], axis = 0)

plausible_entity_ids['plausible_sample'] = 1

subset_data = data.merge(plausible_entity_ids, on = ['entity_id', 'year'], how = 'left')
subset_data['plausible_sample'] = subset_data['plausible_sample'].fillna(0)

subset_data

,entity_id,year,month,financial_report_id,financial_report_template_id,CA,CASH,CL,EQ,NCL,TA,TL,CC,INV,CF_NETTO,CF_SELFFIN,EAT,EBIT,SAL,COST,INT,YIE,LABOR,L3,L2,L1,CF_CL,CASH_TA,SAL_TA,TL_SAL,INV_COST,INV_SAL,CC_SAL,TA_SAL,TL_TA,CF_TL,CL_TA,NCL_TA,EQ_TL,EQ_TA,EBIT_INT,CL_CC,ROE,EAT_TA,ROA_BRUTTO,CF_TA,CF_SAL,ROS,EAT_YIE,ROI,ROA_NETTO,LABOR_PRODUCTIVITY,financial_statement_id,financial_report_order_number,entity_ico,period_length,consolidated,approved_date,type,entity_name,establishment_date,cancellation_date,sknace_code,entity_state,legal_form_code,org_size_code,ownership_category_code,entity_consolidated,sknace_division_name,sknace_division,sknace_subcategory,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,DPHZ_vat_registration_cancelled_3y_off,RU_event,RU_event_1y_off,RU_event_2y_off,RU_event_3y_off,cancellation_year,RUZ_cancelled,RUZ_cancelled_1y_off,RUZ_cancelled_2y_off,RUZ_cancelled_3y_off,sal_class,is_outlier,RUZ_indirect_criteria,RUZ_indirect_criteria_1y_off,RUZ_indirect_criteria_2y_off,RUZ_indirect_criteria_3y_off,combined_issue_criteria_1y_off,combined_issue_criteria_2y_off,combined_issue_criteria_3y_off,plausible_sample
0,4.0,2014.0,1.0,4644773,699.0,8787618.0,5238983.0,4658457.0,2112122.0,3232.0,11200988.0,8657266.0,2578606.0,16429.0,1171078.0,1063283.0,1007897.0,1115692.0,18580340.0,18172171.0,4446.0,19361305.0,3384185.0,1.886380,1.678150,1.124618,0.251388,0.467725,1.658813,0.465937,0.325467,0.318317,0.138781,217.022707,0.772902,0.135271,0.415897,0.000289,0.243971,0.188566,251.942870,1.806580,0.477196,0.089983,0.099607,0.104551,0.063028,0.054245,0.052057,0.090380,0.089983,0.182138,2382229.0,1.0,603783.0,12.0,NaN,2015-07-07,Riadna,"NESS Slovensko, a.s.",1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,2015.0,1.0,5382540,699.0,20276962.0,5687157.0,6153737.0,13229589.0,2890.0,20862929.0,7184264.0,14037315.0,6792.0,3940396.0,3245262.0,6536244.0,7231378.0,17608945.0,11551924.0,2910.0,29573853.0,3581817.0,3.295065,3.205284,0.924179,0.640326,0.272596,0.844030,0.407989,0.211663,0.138857,0.797170,426.524953,0.344355,0.548476,0.294960,0.000139,1.841468,0.634119,2486.009622,0.438384,0.494063,0.313295,0.346614,0.188871,0.223772,0.371189,0.221014,0.313434,0.313295,0.203409,2968194.0,1.0,603783.0,12.0,NaN,2016-09-28,Riadna,"NESS Slovensko, a.s.",1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4.0,2016.0,1.0,5665728,699.0,5991927.0,1828391.0,4859422.0,1178778.0,3568.0,6518310.0,5120402.0,3840942.0,5894.0,247140.0,11702.0,-44282.0,191156.0,12878215.0,12564620.0,38229.0,12914606.0,3476051.0,1.233053,1.166668,0.376257,0.050858,0.280501,1.975698,0.397602,0.168874,0.164762,0.298251,182.214041,0.785541,0.048266,0.745503,0.000547,0.230212,0.180841,6.000288,1.265164,-0.037566,-0.006793,0.029326,0.037915,0.019191,-0.003439,-0.003429,-0.000929,-0.006793,0.269917,3194585.0,3.0,603783.0,12.0,NaN,2017-06-12,Riadna,"NESS Slovensko, a.s.",1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4.0,2017.0,1.0,6245237,699.0,3016170.0,1224056.0,2649399.0,268147.0,3224.0,3442598.0,2902134.0,1355249.0,1954.0,-1327325.0,-1211994.0,-1270731.0,-1386062.0,7651564.0,9093625.0,9037.0,7725983.0,3304021.0,1.138436,0.973543,0.462013,-0.500991,0.355562,2.222613,0.379286,0.077355,0.091934,0.177121,161.971498,0.843007,-0.457362,0.769593,0.000937,0.092396,0.077891,-152.376342,1.954917,-4.738934,-0.369120,-0.402621,-0.385559,-0.173471,-0.166075,-0.164475,-0.366495,-0.369120,0.431810,3662172.0,2.0,603783.0,12.0,NaN,2018-06-26,Riadna,"NESS Slovensko, a.s.",199

In [99]:
subset_data = subset_data.query('plausible_sample == 1 and sal_class not in ["0. <50,000", "4. Veľký"] and period_length == 12 and RUZ_indirect_criteria == 0 and RUZ_cancelled == 0 and RU_event == 0')


In [107]:
master_sample = subset_data.query('RU_event_3y_off == 1').sample(frac=1).groupby('entity_id').first().reset_index()
master_sample_set = set(master_sample['entity_id'])

ruz_cancelled_entity = subset_data.query('RUZ_cancelled_3y_off == 1 and entity_id not in @master_sample_set').sample(frac=1).groupby('entity_id').first().reset_index()

master_sample = pd.concat([master_sample, ruz_cancelled_entity], axis = 0)
master_sample_set = set(master_sample['entity_id'])

ruz_indirect_criteria = subset_data.query('RUZ_indirect_criteria_3y_off == 1 and entity_id not in @master_sample_set').sample(frac=1).groupby('entity_id').first().reset_index()

master_sample = pd.concat([master_sample, ruz_indirect_criteria], axis = 0)
master_sample_set = set(master_sample['entity_id'])

rest = subset_data.query('entity_id not in @master_sample_set').sample(frac=1).groupby('entity_id').first().reset_index()

master_sample = pd.concat([master_sample, rest], axis = 0)

display(master_sample['entity_id'].value_counts())

1471.0       1
615417.0     1
615584.0     1
615564.0     1
615553.0     1
            ..
162555.0     1
162557.0     1
162577.0     1
162593.0     1
1611042.0    1
Name: entity_id, Length: 71046, dtype: int64

In [115]:
for dim in ['RU_event_3y_off', 'RUZ_cancelled_3y_off', 'RUZ_indirect_criteria_3y_off', 'combined_issue_criteria_3y_off']:
    print(dim)
    display(master_sample[dim].value_counts(normalize=True))

RU_event_3y_off


0.0    0.990232
1.0    0.009768
Name: RU_event_3y_off, dtype: float64

RUZ_cancelled_3y_off


0    0.95548
1    0.04452
Name: RUZ_cancelled_3y_off, dtype: float64

RUZ_indirect_criteria_3y_off


0.0    0.83018
1.0    0.16982
Name: RUZ_indirect_criteria_3y_off, dtype: float64

combined_issue_criteria_3y_off


0.0    0.786293
1.0    0.213707
Name: combined_issue_criteria_3y_off, dtype: float64

In [118]:
testing_sample = master_sample.sample(frac=0.2)
testing_sample_ids = set(testing_sample['entity_id'])

In [119]:
training_sample = master_sample.query('entity_id not in @testing_sample_ids')

In [120]:
for dim in ['RU_event_3y_off', 'RUZ_cancelled_3y_off', 'RUZ_indirect_criteria_3y_off', 'combined_issue_criteria_3y_off']:
    print(dim)
    display(
        pd.concat([
            master_sample[dim].value_counts(normalize=True).to_frame('dataset'),
            training_sample[dim].value_counts(normalize=True).to_frame('training_sample'),
            testing_sample[dim].value_counts(normalize=True).to_frame('testing_sample')
        ], axis = 1)
    )

RU_event_3y_off


,dataset,training_sample,testing_sample
0.0,0.990232,0.990306,0.989936
1.0,0.009768,0.009694,0.010064


RUZ_cancelled_3y_off


,dataset,training_sample,testing_sample
0,0.95548,0.95554,0.95524
1,0.04452,0.04446,0.04476


RUZ_indirect_criteria_3y_off


,dataset,training_sample,testing_sample
0.0,0.83018,0.830322,0.829615
1.0,0.16982,0.169678,0.170385


combined_issue_criteria_3y_off


,dataset,training_sample,testing_sample
0.0,0.786293,0.786671,0.784784
1.0,0.213707,0.213329,0.215216


In [121]:
master_sample.to_csv('../../DATA/FINAL/super_sample.csv', index=False)
training_sample.to_csv('../../DATA/FINAL/training_sample.csv', index=False)
testing_sample.to_csv('../../DATA/FINAL/testing_sample.csv', index=False)

In [122]:
for dim in DIMENSIONS:
    print(dim)
    display(
        pd.concat([
            master_sample[dim].value_counts(normalize=True).to_frame('dataset'),
            training_sample[dim].value_counts(normalize=True).to_frame('training_sample'),
            testing_sample[dim].value_counts(normalize=True).to_frame('testing_sample')
        ], axis = 1)
    )

year


,dataset,training_sample,testing_sample
2015.0,0.266546,0.266517,0.266662
2018.0,0.254708,0.254165,0.256879
2016.0,0.242181,0.242518,0.240833
2017.0,0.236565,0.236800,0.235625


period_length


,dataset,training_sample,testing_sample
12.0,1.0,1.0,1.0


type


...

,dataset,training_sample,testing_sample
G,0.281930,0.282175,0.280949
M,0.156828,0.158383,0.150609
C,0.111336,0.110738,0.113731
F,0.092827,0.091842,0.096770
N,0.072432,0.072523,0.072067
Q,0.065929,0.065767,0.066578
L,0.063044,0.062653,0.064607
J,0.050840,0.051234,0.049265
H,0.047350,0.047188,0.047998
I,0.033725,0.033834,0.033289


sal_class


,dataset,training_sample,testing_sample
1. Mikro,0.892731,0.892887,0.892111
2. Malý,0.084452,0.084346,0.084876
3. Stredný,0.022816,0.022767,0.023014


is_outlier


,dataset,training_sample,testing_sample
False,0.988402,0.988282,0.98888
True,0.011598,0.011718,0.01112
